In [1]:
import pandas as pd # For dataset
from pandas.api.types import is_numeric_dtype # For check whether the provided array or dtype is of a numeric dtype.
import xlwt  # For writing to an excel sheet
from xlwt import Workbook # For creating a workbook object

import numpy as np # For computing
from scipy.stats import entropy,itemfreq # For calculating the entropy and array of item frequencies.


df = pd.read_excel (r'personalData.xlsx') # Read dateframe from excel file
print (df) # Print dataset

    Gender       Age Healthy Salary Experience Educational attainment  \
0   female     40-44    good    min     Junior      Bachelor's degree   
1     male     30-34    good    min     Junior      Bachelor's degree   
2     male     25-29    good    min     Junior      Bachelor's degree   
3     male  Under 20    good   high     Senior      Bachelor's degree   
4     male     35-39    good   high     Senior        Master's degree   
..     ...       ...     ...    ...        ...                    ...   
95  female     25-29     bad    min     Junior      Bachelor's degree   
96    male     20-24    good    min     Junior      Bachelor's degree   
97    male  Under 20    good    min     Junior      Bachelor's degree   
98    male     20-24     bad   high     Senior      Bachelor's degree   
99  female     35-39     bad   high     Senior        Master's degree   

             Position Employment type       Working hours  Unnamed: 9  
0           Architect    Student work           Par

In [23]:
# Read metric,threshold and protectedData from textfile

file = open("input.txt","r+") # Open the text file 'input.txt'
lines = file.readlines() # Read data from textfile
file.close()

# Define varibales : metric, threshold and protected data 
metric = '';
threshold= 0;
protected_data = [];

for line in lines:  # Make a loop for a list with using data from text file
    line = line.strip('\n') # Remove both the leading and the trailing characters '\n'
    strings = line.split(": ") # Split a string into a list
    
    # Get the value for metric, threshold and protected data    
    if strings[0].lower() == 'metric': # Check 1st key value of the list with variable (metric)
        metric = strings[1] # If condition is fulfilled then take value of the list for varibale (metric)
    elif strings[0].lower() == 'threshold': # Check 1st key value of the list with variable (threshold)
        threshold= strings[1]   # If condition is fulfilled then take value of the list for varibale (threshold)
    elif strings[0].lower() == 'protected_data': # Check 1st key value of the list with variable (protected_data)
        protected_data = strings[1].split(",") # If condition is fulfilled then take value of the list for varibale (protected_data)

# Print metric, threshold and protected data         
print(metric)
print(threshold)
print(protected_data)

conditionalProbability
0.6
['Gender', 'Age']


In [24]:
# Store data of calculated correlation

diction={}
def storeData(d):
    for item,value in zip(d.index,d.values):  # Use loop for parallel iteration of the result of the calculated correlation
        diction[item[::-1]] = value # Store the result as a dictinary with starting from the end towards the first taking each element.


In [25]:
# Get the only correlated data

dictForHighCorr = {} # Define dictionary for "high" value correlation
def saveResult(d, protected_char, other_att):
    correlation = "low"; # Define string for choosing "high" value correlation
    for item,value in zip(d.index,d.values): # Use loop for parallel iteration
        if float(value) > float(threshold): # Compare threshold from text file with value from dataset
            correlation = "high"  # If value is 'high' then choose them 
    dictForHighCorr[protected_char, other_att] = correlation  # Create dictionary with the choosing data from loop
  

In [26]:
# Calculate the Conditional Entropy of a distribution for given probability values.


# Calculate Entropy (Also known as Shanon Entropy) H(X)
#  Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
def entropy(Y):
   
    
    arr =[] # Define array
    count = itemfreq(Y) # Find unique counts datas related to protected data
   
    
    for x in count:  # Loop for the count of unique value 
        arr.append(x[1]) # Append the count of unique values
    arr = np.array(arr) # Make array list
    
    prob = arr/len(Y) # Calculate probability with using formula: count of unique value of each column / count all value of each column
    en = np.sum((-1)*prob*np.log2(prob)) #  Calculate entropy with using formula: sum ( -1 * probabilty * log2 (probability) 
    return en


# Calculate Joint Entropy H(Y;X)
#  Reference: https://en.wikipedia.org/wiki/Joint_entropy

def jEntropy(Y,X):   
     
    YX = np.c_[Y,X] # Translate slice objects "Protected data" to concatenation along all datas from dataset.
    return entropy(YX) 

# Calculate Conditional Entropy H(Y|X) = H(Y;X) - H(X)
# Reference: https://en.wikipedia.org/wiki/Conditional_entropy

def cEntropy(Y, X):
   
    condEntropy = jEntropy(Y, X) - entropy(X)  # Calculate conditional entropy = Joint Entropy - Entropy of X   
    return condEntropy

In [27]:
#calculate conditional probability of protected_data given other data

def conditionalProbability(data_frame):
    for item in protected_data: # Loop for "Protected data" from text file
        for coli in data_frame: # Loop for all datas from dataset
            
            #this condition prevents calculating CP of protected_data given each other
            if coli.lower() not in [x.lower() for x in protected_data]:  # Check the other data which is not in the "protected data" list
                data = data_frame.groupby(coli)[item].value_counts()/data_frame.groupby(coli)[item].count() # If yes, then calculate CP with formula : count each value of the each column/ count all value of each column
                
                storeData(data) # Store the result in dictinary
                saveResult(data, item, coli) # Save only "high" value correlation
                

In [28]:
calculate conditional entropy of protected_data given other data

def conditionalEntropy(data_frame):   
    corr=''
    df_result = pd.DataFrame([])
    for item in protected_data: # Loop for "Protected data" from text file
        for coli in data_frame: # Loop for all datas from dataset
            
            # this condition prevents calculating CE of protected_data given each other
            if coli.lower() not in [x.lower() for x in protected_data]:  # Check the other data which is not in the "protected data" list
                data = cEntropy(data_frame[item], data_frame[coli]) # Call The CE function with "Protected data" and all data
                if float(data) < float(threshold): # Compare threshold from text file with the result of CE result
                    corr = "high" # If result is true then it's "high"
                else:
                    corr = "low" # If result is false then it's "low" 
                    
                # save result as a dictinary                    
                dictForHighCorr[item, coli] = corr  # Save result as a dictinary 
  

In [29]:
# Check correlation is high or low due to the result   

def checkCorrelation():
    for key in diction: # Make a loop to get key value of the dictionary of calculated correlation 
        if diction[key] > float(threshold): # Compare threshold from text file with key value of the dictionary
              print(key, '->', 'High correlation') # If it's more than threshold , then print "High correlation"
        else:
              print(key, '->', 'Low correlation') # If it's less than threshold , then print "Low correlation"        


In [30]:
# Function to call correlation function regarding measure method

def correlationMethod(metric,data_frame):
    if metric=='conditionalProbability': # Check if measure method is 'conditionalProbability'
        conditionalProbability(data_frame) # Show the result of correlation function
        diction
    elif metric=='conditionalEntropy': # Check if measure method is 'conditionalEntropy'
        conditionalEntropy(data_frame) # Show the result of correlation function
    else: # If measure method is't defined then print this message
        print("Correlation with this measurement method is not calculated!")


In [31]:
correlationMethod(metric,df) # Call the function for calculation method

In [32]:
checkCorrelation() # Get the result with respect to value of each 'protected data'

('female', 'bad') -> Low correlation
('male', 'bad') -> Low correlation
('male', 'good') -> Low correlation
('female', 'good') -> Low correlation
('female', 'high') -> High correlation
('male', 'high') -> Low correlation
('male', 'med') -> High correlation
('female', 'med') -> Low correlation
('male', 'min') -> Low correlation
('female', 'min') -> Low correlation
('female', 'Expert') -> High correlation
('male', 'Expert') -> Low correlation
('male', 'Junior') -> Low correlation
('female', 'Junior') -> Low correlation
('female', 'Senior') -> Low correlation
('male', 'Senior') -> Low correlation
('male', "Bachelor's degree") -> Low correlation
('female', "Bachelor's degree") -> Low correlation
('female', 'Doctoral degree') -> High correlation
('male', 'Doctoral degree') -> Low correlation
('male', "Master's degree") -> Low correlation
('female', "Master's degree") -> Low correlation
('male', 'Some college, no degree') -> Low correlation
('female', 'Some college, no degree') -> Low correl

In [33]:
dictForHighCorr = {k: v for k, v in dictForHighCorr.items() if v != 'low'} # Get the result only for "high" correlation

In [34]:
dictForHighCorr # Print the result

{('Gender', 'Salary'): 'high',
 ('Gender', 'Experience'): 'high',
 ('Gender', 'Educational attainment'): 'high',
 ('Gender', 'Position'): 'high',
 ('Gender', 'Employment type'): 'high',
 ('Gender', 'Working hours'): 'high',
 ('Age', 'Position'): 'high'}

In [35]:
if not bool(dictForHighCorr): #  Check the result, if there is't any result, then print message
    print("No correlation!")
else: # If there is some result then export the result as a excell file
    df = pd.DataFrame(data=dictForHighCorr, index=['0']) # Take the data with index = 0
    df = (df.T) # Transpose the data frame
    df = df.iloc[:, :-1] # Select till the second last column of the data frame 
    df
    df.to_excel('dict1.xlsx') # Write result to excel file 
